# start

In [1]:
TGT='Reply'
PRFX='Reply0521A2'

trntmstmp=1584412344
tsttmstmp=1590075806

tm_trn_min,tm_trn_max=(1580947200, 1581551999)
tm_tst_min,tm_tst_max=(1581552000, 1582156799)

train_total=148075238
test_total=int(1.4e7)
print(f'train_total {train_total:,}, test_total {test_total:,}')

SEED=101

trnsz = int(1e7)
valsz = trnsz//10
print(f'train_total {trnsz:,}, valsz {valsz:,}')



import datetime
def showtm(tm): return datetime.datetime.fromtimestamp(tm).strftime('%Y-%m-%d %H:%M:%S')
print([showtm(tm) for tm in (trntmstmp, tsttmstmp)])
print([showtm(tm) for tm in (tm_trn_min,tm_trn_max)])
print([showtm(tm) for tm in (tm_tst_min,tm_tst_max)])

train_total 148,075,238, test_total 14,000,000
train_total 10,000,000, valsz 1,000,000
['2020-03-17 02:32:24', '2020-05-21 15:43:26']
['2020-02-06 00:00:00', '2020-02-12 23:59:59']
['2020-02-13 00:00:00', '2020-02-19 23:59:59']


# setup

In [2]:
from tqdm import tqdm
import time
from pathlib import Path
from collections import Counter,defaultdict
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve
import pickle
# import xgboost as xgb
import lightgbm as lgb
import category_encoders as ce
import gc

import optuna
# optuna.logging.CRITICAL, optuna.logging.FATAL
# optuna.logging.ERROR
# optuna.logging.WARNING, optuna.logging.WARN
# optuna.logging.INFO
# optuna.logging.DEBUG
# optuna.logging.set_verbosity(optuna.logging.ERROR)
import optuna.integration.lightgbm as lgb
optuna.logging.disable_default_handler()


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

def iou(s1,s2):
    return len(s1&s2) / len(s1|s2)

HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
p_out=f'{HOME}/output/{PRFX}'
Path(p_out).mkdir(exist_ok=True, parents=True)

from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

# https://towardsdatascience.com/how-to-calibrate-undersampled-model-scores-8f3319c1ea5b
# How to use the function?
# Let’s say your goal is to generate a model that shows the credit default probabilities and your original 
# training data has 50,000 rows with only 500 of them labeled as target class. When you sample your non-target 
# instances randomly and reduce the total row count to 10,000, while conserving 500 target rows, our calibration
# function becomes:
# calibration(model_results, 50000, 500, 10000, 500)
# Here model_results is your model probability output array. After you train your model and put the results in it, your function is ready to use. 
def calibration(data, train_pop, target_pop, sampled_train_pop, sampled_target_pop):
    calibrated_data = ((data * (target_pop / train_pop) / (sampled_target_pop / sampled_train_pop)) /
    (((1 - data) * (1 - target_pop / train_pop) / (1 - sampled_target_pop / sampled_train_pop)) +
     (data * (target_pop / train_pop) / (sampled_target_pop / sampled_train_pop))))
    return calibrated_data


cols=['toks','hshtgs','twtid','media','links','domns','twttyp','lang','tm','u1id','u1_fllwer_cnt','u1_fllwing_cnt','u1_vrfed','u1_create_tm','u2id','u2_fllwer_cnt','u2_fllwng_cnt','u2_vrfed','u2_create_tm','u1_fllw_u2','reply_tm','retwt_tm','retwt_cmmnt_tm','like_tm',]

cols_val = cols[:-4]
cols_tgt_tmstmp=[ 'retwt_tm', 'reply_tm', 'like_tm', 'retwt_cmmnt_tm',]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
tgt2col = dict(zip(tgts, cols_tgt))
TCOL=tgt2col[TGT]
print('TCOL', TCOL)
print(tgt2col)

TCOL reply
{'Retweet': 'retwt', 'Reply': 'reply', 'Like': 'like', 'RTwCmnt': 'retwt_cmmnt'}


# prep

In [3]:
def prp_tgt(df):
    df[cols_tgt]=df[cols_tgt_tmstmp].notna()
    df.drop(columns=cols_tgt_tmstmp, inplace=True)
    display(df[cols_tgt].mean())

In [4]:
%%time
dfval = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',header=None,names=cols,nrows=valsz)
print(dfval.shape)
prp_tgt(dfval)

(1000000, 24)


retwt          0.113135
reply          0.027688
like           0.438804
retwt_cmmnt    0.007750
dtype: float64

CPU times: user 11.2 s, sys: 1.36 s, total: 12.6 s
Wall time: 25.2 s


## maybe oversample

In [5]:
# df = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',header=None,names=cols,nrows=5)
# dfval = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',header=None,names=cols,nrows=2)
# dftrn = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',header=None,names=cols,skiprows=2,nrows=3)
# display(df.twtid)
# display(dfval.twtid)
# display(dftrn.twtid)

# 0    D4D1EBDE74F74C5DA529959AF979625C
# 1    BFB529DAB6D384EB83E899A72AB3830D
# 2    519078C7834E9642508F72A6C2D0F3B7
# 3    52AAE9E33EFAC8C478C57B31A9E31ED1
# 4    89C1298C55EB3D68E2784F0BFB69E6F8
# Name: twtid, dtype: object
# 0    D4D1EBDE74F74C5DA529959AF979625C
# 1    BFB529DAB6D384EB83E899A72AB3830D
# Name: twtid, dtype: object
# 0    519078C7834E9642508F72A6C2D0F3B7
# 1    52AAE9E33EFAC8C478C57B31A9E31ED1
# 2    89C1298C55EB3D68E2784F0BFB69E6F8
# Name: twtid, dtype: object

In [6]:
%%time
if TGT in ('Retweet','Like'): trnpop=trnsz
if TGT=='Reply': trnpop=trnsz*3.7
if TGT=='RTwCmnt': trnpop=trnsz*11.5
assert trnpop<train_total

dftrn = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',header=None,names=cols,
                     skiprows=valsz, nrows=trnpop)
print(dftrn.shape)
prp_tgt(dftrn)

(37000000, 24)


retwt          0.113052
reply          0.027434
like           0.439190
retwt_cmmnt    0.007789
dtype: float64

CPU times: user 8min 11s, sys: 2min, total: 10min 12s
Wall time: 20min 56s


In [7]:
train_pop=len(dftrn)
sampled_train_pop=trnsz
idx_pos = np.where(dftrn[TCOL])[0]
target_pop=sampled_target_pop=len(idx_pos)
if TGT in ('Reply','RTwCmnt'):
    idx_neg0 = np.where(~dftrn[TCOL])[0]
    idx_neg = np.random.choice(idx_neg0, trnsz-len(idx_pos), replace=False)
    idx = np.concatenate([idx_pos,idx_neg])
    dftrn = dftrn.iloc[idx].copy()
print(dftrn.shape, dftrn[TCOL].mean())
print(train_pop, target_pop, sampled_train_pop, sampled_target_pop)

display(dftrn.head())
display(dftrn.columns)

(10000000, 24) 0.1015047
37000000 1015047 10000000 1015047


toks hshtgs  \
11   101\t22734\t19540\t10393\t124\t76456\t22201\t1...    NaN   
32   101\t100397\t11447\t119\t100397\t11447\t119\t1...    NaN   
67                  101\t232\t10113\t44730\t17828\t102    NaN   
84   101\t10862\t11710\t11223\t14517\t89106\t17021\...    NaN   
118  101\t4726\t4997\t3740\t1931\t26164\t29570\t706...    NaN   

                                twtid media links domns    twttyp  \
11   F11BD399C084D3035209F62EC72C0E6A   NaN   NaN   NaN  TopLevel   
32   65AFF257ED64E9FA0428A761AA305287   NaN   NaN   NaN     Quote   
67   96D790B62A2C15769B830E3A3E51BF34   NaN   NaN   NaN  TopLevel   
84   46FB82A9A3034DDBD4A7C1E545720DF0   NaN   NaN   NaN  TopLevel   
118  9F3630A26084A55B8326376FC4AF5AF2   NaN   NaN   NaN  TopLevel   

                                 lang          tm  \
11   D3164C7FBCF2565DDF915B1B3AEFB1DC  1581044002   
32   1FFD2FE4297F5E70EBC6C3230D95CB9C  1581462579   
67   FA3F382BC409C271E3D6EAF8BE4648DD  1581374220   
84   D3164C7FBCF2565DDF915B1B3AEFB1DC  1581536890   
118  22C448FF81263D4BAF2A176145EE9EAD  1581463772   

                                 u1id  u1_fllwer_cnt  u1_fllwing_cnt  \
11   CFF9C8C70792DC31C769F29CBFA2FEBE            972             811   
32   5BA44B16761DE43723E20E995437805C            514             858   
67   2111169D09C052121BDF329807DC75A4           5296            1088   
84   F8742926680CE7BA15DA13CE557BE011           2234             416   
118  FE2B80E908ED0EF93CE0749BD93854BF            128             144   

     u1_vrfed  u1_create_tm                              u2id  u2_fllwer_cnt  \
11      False    1575970944  0DD2B48B86395C04C36C377F0F499CC4            741   
32      False    1237411551  0DD2CF17568371669DD5807061A7F231             17   
67      False    1286175742  0DD2ED2C408A60972110349BC121AEA7             77   
84      False    1305458959  0DD2FD96A5F56E0ABA54276D3CCC7480           1178   
118     False    1567089785  0DD31F56720FD03FF243779A0004A4A8            146   

     u2_fllwng_cnt  u2_vrfed  u2_create_tm  u1_fllw_u2  retwt  reply   like  \
11             736     False    1563093560        True  False   True   True   
32              38     False    1376576148        True  False   True  False   
67             304     False    1369165590       False  False   True  False   
84            1661     False    1349738202       False  False   True  False   
118            191     False    1571151626        True  False   True  False   

     retwt_cmmnt  
11         False  
32         False  
67         False  
84         False  
118        False

Index(['toks', 'hshtgs', 'twtid', 'media', 'links', 'domns', 'twttyp', 'lang',
       'tm', 'u1id', 'u1_fllwer_cnt', 'u1_fllwing_cnt', 'u1_vrfed',
       'u1_create_tm', 'u2id', 'u2_fllwer_cnt', 'u2_fllwng_cnt', 'u2_vrfed',
       'u2_create_tm', 'u1_fllw_u2', 'retwt', 'reply', 'like', 'retwt_cmmnt'],
      dtype='object')

## prep and features

In [8]:
def prp_df(df, tm_max):
    '''
    tm_max = tm_trn_max for train tm_tst_max for test
    '''
    lendf = len(df)
    print(dtnow(), 'start')
    set_u1 = set(df.u1id)
    set_u2 = set(df.u2id)
    df['u1inu2']=df.u1id.isin(set_u2)
    df['u2inu1']=df.u2id.isin(set_u1)

    #counts
    df['len_toks'] = df.toks.apply(lambda x: len(x.split('\t')))
    df.drop(columns=['toks'], inplace=True)    
    for m in ['Photo', 'Video', 'GIF']:
        df[f'n_media_{m}'] = df['media'].fillna('').apply(lambda x: x.split('\t').count(m))
    tgnms = ['hshtgs','media', 'links', 'domns',]
    for tgnm in tgnms:
        df[f'has_{tgnm}']=df[tgnm].notna()
        df[f'lst_{tgnm}'] = df[tgnm].fillna('').apply(lambda x: x.split('\t') if len(x) else [])
        df[f'n_{tgnm}'] = df[f'lst_{tgnm}'].apply(len)  
    df.drop(columns=['has_links','n_links'], inplace=True) #duplicates has_domns,n_domns 
    
    #time
    dt = pd.to_datetime(df.tm, unit='s')
    df['dayofweek'] = dt.dt.dayofweek
    df['hour'] = dt.dt.hour
    
    df['tmdlta_u2u1']  = df.u2_create_tm - df.u1_create_tm
    df['tmdlta_twtu1'] = df.tm - df.u1_create_tm
    df['tmdlta_twtu2'] = df.tm - df.u2_create_tm

    df['twt_age']   = tm_max-df.tm
    df['u1_age']    = tm_max-df.u1_create_tm
    df['u2_age']    = tm_max-df.u2_create_tm
    df.drop(columns=['tm', 'u1_create_tm', 'u2_create_tm'], inplace=True)

    #time derived
    df['u1_fllwer_cnt_by_age'] = df.u1_fllwer_cnt / df.u1_age
    df['u1_fllwng_cnt_by_age'] = df.u2_fllwng_cnt / df.u2_age
    
    #interaction
    df['u1u2']=df.u1id+'_'+df.u2id
    df['langhour']=df.lang+'_'+df.hour.astype(str)
    

#     #freq of feature values
#     print(dtnow(), 'freq of columns using CountEncoder')
#     encoder = ce.CountEncoder()
#     encoded = encoder.fit_transform(
#     df[['twtid', 'twttyp', 'lang', 'u1id', 'u1_fllwing_cnt', 'u1_vrfed', 'u2id', 'u2_fllwer_cnt', 'u2_vrfed', 'u1_fllw_u2', 
#        'n_media_Photo', 'n_media_Video', 'n_media_GIF', 
#        'has_hshtgs', 'n_hshtgs', 'has_media', 'n_media','has_domns', 'n_domns', 
#        'dayofweek', 'hour', 'u1u2', 'langhour']].astype(object))
#     encoded = encoded.astype(int)/lendf
#     encoded.columns = [f'frq_{col}' for col in encoded.columns]
#     df = pd.concat([df,encoded],1)
    
    #freq of tgnm values
#     print(dtnow(), 'freq of tags')
#     for tgnm in tgnms:
#         vs = [j for i in df[f'lst_{tgnm}'] for j in i]
#         cnt = Counter(vs)
#         frq = {k:v/lendf for k,v in cnt.items()}
#         df[f'sumfrq_{tgnm}']=df[f'lst_{tgnm}'].apply(lambda x: sum([frq.get(o,0) for o in x]))
#         df[f'maxfrq_{tgnm}']=df[f'lst_{tgnm}'].apply(lambda x: max([frq.get(o,0) for o in x]) if len(x) else 0)
    
    df.drop(columns=tgnms+[f'lst_{tgnm}' for tgnm in tgnms],inplace=True)

    print(dtnow(), 'done')
    return df

In [9]:
%%time
dftrn=prp_df(dftrn, tm_max=tm_trn_max)

2020-05-21 20:51:04 start
2020-05-21 21:11:38 done
CPU times: user 9min 49s, sys: 2min 12s, total: 12min 1s
Wall time: 20min 37s


In [10]:
%%time
dfval=prp_df(dfval, tm_max=tm_trn_max)

2020-05-21 21:11:41 start
2020-05-21 21:11:57 done
CPU times: user 14.1 s, sys: 2.26 s, total: 16.3 s
Wall time: 16.1 s


## tr vl split

In [11]:
msk_vl=np.random.rand(len(dftrn))<0.15
idxvl=np.where( msk_vl)[0]
idxtr=np.where(~msk_vl)[0]

dftr = dftrn.iloc[idxtr].copy()
dfvl = dftrn.iloc[idxvl].copy()
del dftrn

gc.collect()

30

In [12]:
cols_category=['twttyp', 'lang', 'langhour']
for df in dftr,dfvl,dfval:
    df.drop(columns=['twtid','u1id','u2id','u1u2'], inplace=True)
    df[cols_category]=df[cols_category].astype('category')

In [13]:
cols_feat = [col for col in dftr.columns if col not in cols_tgt]

# model

## cols_feat

In [14]:
cols_feat

['twttyp',
 'lang',
 'u1_fllwer_cnt',
 'u1_fllwing_cnt',
 'u1_vrfed',
 'u2_fllwer_cnt',
 'u2_fllwng_cnt',
 'u2_vrfed',
 'u1_fllw_u2',
 'u1inu2',
 'u2inu1',
 'len_toks',
 'n_media_Photo',
 'n_media_Video',
 'n_media_GIF',
 'has_hshtgs',
 'n_hshtgs',
 'has_media',
 'n_media',
 'has_domns',
 'n_domns',
 'dayofweek',
 'hour',
 'tmdlta_u2u1',
 'tmdlta_twtu1',
 'tmdlta_twtu2',
 'twt_age',
 'u1_age',
 'u2_age',
 'u1_fllwer_cnt_by_age',
 'u1_fllwng_cnt_by_age',
 'langhour']

In [15]:
print(len(cols_feat))


32


In [16]:
results = {}
results['PRFX'] = PRFX
results['TGT'] = TGT
results['cols_feat'] = cols_feat

In [17]:
params={
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbosity': 0,    
}

results['params'] = params

In [ ]:
%%time
dtr = lgb.Dataset(dftr[cols_feat], label=dftr[TCOL])
dvl = lgb.Dataset(dfvl[cols_feat], label=dfvl[TCOL])
best_params, tuning_history = {}, []
evalres = {}
evallist = [(dtr, 'train'), (dvl, 'eval')]
bst = lgb.train(params=params, 
                train_set=dtr, 
                num_boost_round=50000,
                valid_sets=[dtr, dvl],
                valid_names=['tr','vl'],
                best_params=best_params,
                tuning_history=tuning_history,
                verbose_eval=100,
                early_stopping_rounds=100,
                evals_result=evalres,
               )
results['bst'] = bst
results['evalres'] = evalres
results['best_params']=best_params
results['tuning_history']=tuning_history

/data/anaconda3/envs/rcss20/lib/python3.7/site-packages/optuna/_experimental.py:87: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning
tune_feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[100]	tr's binary_logloss: 0.279099	vl's binary_logloss: 0.279454
[200]	tr's binary_logloss: 0.277208	vl's binary_logloss: 0.278022
[300]	tr's binary_logloss: 0.276059	vl's binary_logloss: 0.277302
[400]	tr's binary_logloss: 0.275265	vl's binary_logloss: 0.276864
[500]	tr's binary_logloss: 0.274545	vl's binary_logloss: 0.276537
[600]	tr's binary_logloss: 0.273903	vl's binary_logloss: 0.276277
[700]	tr's binary_logloss: 0.273274	vl's binary_logloss: 0.276025
[800]	tr's binary_logloss: 0.272794	vl's binary_logloss: 0.275894
[900]	tr's binary_logloss: 0.272286	vl's binary_logloss: 0.275682
[1000]	tr's binary_logloss: 0.2718	vl's binary_logloss: 0.275525
[1100]	tr's binary_logloss: 0.271366	vl's binary_logloss: 0.275411
[1200]	tr's binary_logloss: 0.270929	vl's binary_logloss: 0.275272
[1300]	tr's binary_logloss: 0.27054	vl's binary_logloss: 0.275178
[1400]	tr's binary_logloss: 0.270149	vl's binary_logloss: 0.275089
[1500]	tr's 

tune_feature_fraction, val_score: 0.274075:  14%|#4        | 1/7 [43:47<4:22:44, 2627.38s/it]

Training until validation scores don't improve for 100 rounds
[100]	tr's binary_logloss: 0.278671	vl's binary_logloss: 0.279108
[200]	tr's binary_logloss: 0.276816	vl's binary_logloss: 0.277815
[300]	tr's binary_logloss: 0.275778	vl's binary_logloss: 0.277268
[400]	tr's binary_logloss: 0.274789	vl's binary_logloss: 0.276772
[500]	tr's binary_logloss: 0.274007	vl's binary_logloss: 0.276455
[600]	tr's binary_logloss: 0.273323	vl's binary_logloss: 0.276192
[700]	tr's binary_logloss: 0.272717	vl's binary_logloss: 0.275983
[800]	tr's binary_logloss: 0.272115	vl's binary_logloss: 0.275806
[900]	tr's binary_logloss: 0.271605	vl's binary_logloss: 0.275636
[1000]	tr's binary_logloss: 0.271149	vl's binary_logloss: 0.275545
[1100]	tr's binary_logloss: 0.270671	vl's binary_logloss: 0.275391
[1200]	tr's binary_logloss: 0.270229	vl's binary_logloss: 0.275299
[1300]	tr's binary_logloss: 0.269835	vl's binary_logloss: 0.275216
[1400]	tr's binary_logloss: 0.26942	vl's binary_logloss: 0.275131
[1500]	tr'

tune_feature_fraction, val_score: 0.274075:  29%|##8       | 2/7 [1:11:05<3:14:12, 2330.43s/it]

...

Training until validation scores don't improve for 100 rounds
[100]	tr's binary_logloss: 0.273774	vl's binary_logloss: 0.276233
[200]	tr's binary_logloss: 0.270944	vl's binary_logloss: 0.275147
[300]	tr's binary_logloss: 0.269059	vl's binary_logloss: 0.274744
[400]	tr's binary_logloss: 0.267458	vl's binary_logloss: 0.274515
[500]	tr's binary_logloss: 0.265941	vl's binary_logloss: 0.274381
[600]	tr's binary_logloss: 0.264484	vl's binary_logloss: 0.27425
[700]	tr's binary_logloss: 0.263046	vl's binary_logloss: 0.274118
[800]	tr's binary_logloss: 0.261622	vl's binary_logloss: 0.274045
[900]	tr's binary_logloss: 0.2604	vl's binary_logloss: 0.274015
[1000]	tr's binary_logloss: 0.259082	vl's binary_logloss: 0.274006
[1100]	tr's binary_logloss: 0.257808	vl's binary_logloss: 0.273981
[1200]	tr's binary_logloss: 0.256594	vl's binary_logloss: 0.273926
[1300]	tr's binary_logloss: 0.255369	vl's binary_logloss: 0.273874
[1400]	tr's binary_logloss: 0.254307	vl's binary_logloss: 0.273836
[1500]	tr's 

tune_lambda_l1_and_lambda_l2, val_score: 0.273287:  35%|###5      | 7/20 [6:53:10<12:38:34, 3501.13s/it]

Training until validation scores don't improve for 100 rounds
[100]	tr's binary_logloss: 0.273969	vl's binary_logloss: 0.276333
[200]	tr's binary_logloss: 0.271238	vl's binary_logloss: 0.275175
[300]	tr's binary_logloss: 0.269536	vl's binary_logloss: 0.274668
[400]	tr's binary_logloss: 0.268079	vl's binary_logloss: 0.274392
[500]	tr's binary_logloss: 0.266707	vl's binary_logloss: 0.274169
[600]	tr's binary_logloss: 0.265287	vl's binary_logloss: 0.273959
[700]	tr's binary_logloss: 0.264179	vl's binary_logloss: 0.273842
[800]	tr's binary_logloss: 0.263221	vl's binary_logloss: 0.273778
[900]	tr's binary_logloss: 0.262157	vl's binary_logloss: 0.273672
[1000]	tr's binary_logloss: 0.261154	vl's binary_logloss: 0.273643
[1100]	tr's binary_logloss: 0.260284	vl's binary_logloss: 0.273556
[1200]	tr's binary_logloss: 0.259289	vl's binary_logloss: 0.27349
[1300]	tr's binary_logloss: 0.258379	vl's binary_logloss: 0.273445
[1400]	tr's binary_logloss: 0.257374	vl's binary_logloss: 0.273413
[1500]	tr'

tune_lambda_l1_and_lambda_l2, val_score: 0.273287:  40%|####      | 8/20 [8:03:20<12:22:45, 3713.78s/it]

Training until validation scores don't improve for 100 rounds
[100]	tr's binary_logloss: 0.273832	vl's binary_logloss: 0.276213
[200]	tr's binary_logloss: 0.271034	vl's binary_logloss: 0.27515
[300]	tr's binary_logloss: 0.26906	vl's binary_logloss: 0.274745


## best_params

In [39]:
best_params

{'lambda_l1': 7.908398474455116,
 'lambda_l2': 0.0082237391930867,
 'num_leaves': 170,
 'feature_fraction': 0.41600000000000004,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20}

## tr vl trajec

In [40]:
plt.plot(evalres['tr']['binary_logloss'])
plt.plot(evalres['vl']['binary_logloss'])
plt.title(f"logloss; best_iteration {bst.best_iteration}")
plt.show()

## model features

In [41]:
ax = lgb.plot_importance(bst, height=0.8, max_num_features=50, figsize=(10,15))
ax.grid(False, axis="y")
ax.set_title(f'Estimated feature importance')
plt.show()

In [42]:
pd.DataFrame([bst.feature_name(), bst.feature_importance()]).T.sort_values(1, ascending=False)

0      1
31              langhour  23217
3         u1_fllwing_cnt  21745
29  u1_fllwer_cnt_by_age  20023
5          u2_fllwer_cnt  19437
11              len_toks  19250
2          u1_fllwer_cnt  18890
26               twt_age  18493
24          tmdlta_twtu1  18219
6          u2_fllwng_cnt  17854
30  u1_fllwng_cnt_by_age  17268
23           tmdlta_u2u1  17144
27                u1_age  16311
25          tmdlta_twtu2  15526
28                u2_age  15301
1                   lang  10650
22                  hour   6967
21             dayofweek   4187
0                 twttyp   4095
16              n_hshtgs   3331
12         n_media_Photo   2707
18               n_media   2179
20               n_domns   1084
9                 u1inu2    855
13         n_media_Video    671
19             has_domns    632
15            has_hshtgs    625
10                u2inu1    571
17             has_media    562
8             u1_fllw_u2    538
4               u1_vrfed    502
14           n_media_GIF     79
7               u2_vrfed     48

In [43]:
col2imp = dict(zip(bst.feature_name(), bst.feature_importance()))
sorted(col2imp.items(), key=lambda x: -x[1])

[('langhour', 23217),
 ('u1_fllwing_cnt', 21745),
 ('u1_fllwer_cnt_by_age', 20023),
 ('u2_fllwer_cnt', 19437),
 ('len_toks', 19250),
 ('u1_fllwer_cnt', 18890),
 ('twt_age', 18493),
 ('tmdlta_twtu1', 18219),
 ('u2_fllwng_cnt', 17854),
 ('u1_fllwng_cnt_by_age', 17268),
 ('tmdlta_u2u1', 17144),
 ('u1_age', 16311),
 ('tmdlta_twtu2', 15526),
 ('u2_age', 15301),
 ('lang', 10650),
 ('hour', 6967),
 ('dayofweek', 4187),
 ('twttyp', 4095),
 ('n_hshtgs', 3331),
 ('n_media_Photo', 2707),
 ('n_media', 2179),
 ('n_domns', 1084),
 ('u1inu2', 855),
 ('n_media_Video', 671),
 ('has_domns', 632),
 ('has_hshtgs', 625),
 ('u2inu1', 571),
 ('has_media', 562),
 ('u1_fllw_u2', 538),
 ('u1_vrfed', 502),
 ('n_media_GIF', 79),
 ('u2_vrfed', 48)]

In [44]:
[o[0] for o in sorted(col2imp.items(), key=lambda x: -x[1]) if o[1]>0]

['langhour',
 'u1_fllwing_cnt',
 'u1_fllwer_cnt_by_age',
 'u2_fllwer_cnt',
 'len_toks',
 'u1_fllwer_cnt',
 'twt_age',
 'tmdlta_twtu1',
 'u2_fllwng_cnt',
 'u1_fllwng_cnt_by_age',
 'tmdlta_u2u1',
 'u1_age',
 'tmdlta_twtu2',
 'u2_age',
 'lang',
 'hour',
 'dayofweek',
 'twttyp',
 'n_hshtgs',
 'n_media_Photo',
 'n_media',
 'n_domns',
 'u1inu2',
 'n_media_Video',
 'has_domns',
 'has_hshtgs',
 'u2inu1',
 'has_media',
 'u1_fllw_u2',
 'u1_vrfed',
 'n_media_GIF',
 'u2_vrfed']

# eval

In [ ]:
%%time
prdtr = bst.predict(dftr[cols_feat],num_iteration=bst.best_iteration)
prdvl = bst.predict(dfvl[cols_feat],num_iteration=bst.best_iteration)
prdval0 = bst.predict(dfval[cols_feat],num_iteration=bst.best_iteration)
prdval = calibration(prdval0, train_pop, target_pop, sampled_train_pop, sampled_target_pop)

In [ ]:
%%time
ytr,yvl,yval=[df[TCOL].values for df in (dftr,dfvl,dfval)]

auc_tr=compute_prauc(prdtr, ytr)
rce_tr=compute_rce(prdtr, ytr)
auc_vl=compute_prauc(prdvl, yvl)
rce_vl=compute_rce(prdvl, yvl)
auc_val=compute_prauc(prdval, yval)
rce_val=compute_rce(prdval, yval)

results['scrs'] = {}
results['scrs']['auc_tr']=auc_tr
results['scrs']['rce_tr']=rce_tr
results['scrs']['auc_vl']=auc_vl
results['scrs']['rce_vl']=rce_vl
results['scrs']['auc_val']=auc_val
results['scrs']['rce_val']=rce_val

In [38]:
print(f'auc_tr:  {auc_tr:.4f}')
print(f'auc_vl:  {auc_vl:.4f}')
print(f'auc_val: {auc_val:.4f}')
print()
print(f'rce_tr:  {rce_tr:.4f}')
print(f'rce_vl:  {rce_vl:.4f}')
print(f'rce_val: {rce_val:.4f}')

auc_tr:  0.4149
auc_vl:  0.3247
auc_val: 0.1178

rce_tr:  22.4319
rce_vl:  16.8343
rce_val: 13.0807


## score

In [37]:
print(f'-{auc_val:.4f}-{rce_val:.4f}')

-0.1178-13.0807


# save results

In [ ]:
pickle.dump(results, open(f'{p_out}/results_{PRFX}.p', 'wb'))

# infer

In [ ]:
results = pickle.load( open(f'{p_out}/results_{PRFX}.p', 'rb'))
bst = results['bst']
cols_feat = results['cols_feat']

## prep

In [ ]:
%%time
dftst=pd.read_csv(
    f'{p_in}/val_{tsttmstmp}.tsv',
    sep='\x01', header=None, names=cols_val, 
#     nrows=10
)

In [ ]:
dftst.shape

In [ ]:
%%time
dftst=prp_df(dftst, tm_max=tm_tst_max)

In [ ]:
dftst.drop(columns=['u1id','u1u2'], inplace=True)
cols_category=['twttyp', 'lang', 'langhour']
dftst[cols_category]=dftst[cols_category].astype('category')

## predict

In [ ]:
%%time
prdtst0 = bst.predict(dftst[cols_feat],num_iteration=bst.best_iteration)
prdtst = calibration(prdtst0, train_pop, target_pop, sampled_train_pop, sampled_target_pop)

In [ ]:
dfsub_ids = dftst[['twtid','u2id',]]
dfsub = dfsub_ids.copy()
dfsub['scr'] = prdtst
dfsub.to_csv(f'{p_out}/{TGT}__{PRFX}.csv',index=False,header=False)